In [3]:
import numpy as np
import pandas as pd

# 读取三张表格

In [4]:
prob_matching = pd.read_csv('./csv/df_al_block200.csv',
                                header=0)
print(prob_matching.columns)

Index(['room_id', 'room_id.block', 'room_id.block_id', 'mobiles.mobile',
       'mobiles.score', 'mobiles.score1', 'mobiles.score2',
       'mobiles.plan_months', 'mobiles.app_price', 'mobiles.dx_price',
       'mobiles.price', 'mobiles.app_poiblock1', 'mobiles.app_poiblock2',
       'mobiles.poiblock1', 'mobiles.poiblock2', 'mobiles.rent_type',
       'mobiles.dx_rent_type', 'str', 'judge'],
      dtype='object')


In [5]:
sub2sub = pd.read_csv('./csv/subway_station.csv',
                          header=None,
                          names=['sub1','sub2','nums'])
print(sub2sub.columns)

Index(['sub1', 'sub2', 'nums'], dtype='object')


In [6]:
sub_info = pd.read_csv('./csv/subways.csv',
                           header=None,
                           names = ['id', 'city', 'name', 'lng_lat', 'lines', 'nearby', 'transference',
                                   'created_at', 'updated_at', 'construction_status', 'open_time','status'])

sub_info = sub_info[1:][['id','name','lng_lat']][sub_info['city']=='北京市']
print(sub_info.columns)

Index(['id', 'name', 'lng_lat'], dtype='object')


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


# 功能函数
## 逻辑：
        根据推荐的商圈名称找到对应的地铁id
        在根据room_id和刚刚查到的地铁id，找到两个地铁之间的站数

In [86]:
# 通过商圈名称找到地铁id
def get_both_id(prob_matching,sub_info):
    block_id_lst = []
    mobiles_poiblock1_id_lst = []
    # 查找每个mobile.poiblock1的对应id
    for index,i in enumerate(range(len(prob_matching))):
        block = prob_matching.loc[i,]['room_id.block']
        block_id = sub_info[sub_info['name']==block]['id'].values[0]
        try:
            # 得到每条数据的推荐商圈名，根据名称去sub_info里面查找
            mobiles_poiblock1 = prob_matching.loc[i,]['mobiles.poiblock1']
            mobiles_poiblock1_id = sub_info[sub_info['name']==mobiles_poiblock1]['id'].values[0]
        except:
            # 填写非地铁站用mobiles.app_poiblock1中代替,例如大运河，都查不到则用*替代
            mobiles_poiblock1 = prob_matching.loc[i,]['mobiles.app_poiblock1']
            tmp = sub_info[sub_info['name']==mobiles_poiblock1]['id']
            # 虎泉 测试
            if len(tmp)==0:
#                 continue
                mobiles_poiblock1_id = '*'
            else:
                mobiles_poiblock1_id = tmp.values[0]
#         print(mobiles_poiblock1,mobiles_poiblock1_id)
        block_id_lst.append(block_id)
        mobiles_poiblock1_id_lst.append(mobiles_poiblock1_id)
    assert len(block_id_lst)==len(mobiles_poiblock1_id_lst),'BUG IN get_both_id()!'
    return zip(block_id_lst,mobiles_poiblock1_id_lst)

In [93]:
# 通过两个id来找到得到的subs_num
def get_subs_num(ziped_id,sub2sub):
    subs_num_lst = []
    for x,y in ziped_id:
        # 通过两个id进行查找
        print(x,y)
        if x==y:
            subs_num = 0
        elif x=='*' or y=='*':
            subs_num = 'subways中未查到地铁'
        else:
            tmp = sub2sub[(sub2sub['sub1']==int(x)) & (sub2sub['sub2']==int(y))]
            if len(tmp)==0:
                tmp = sub2sub[(sub2sub['sub1']==int(y)) & (sub2sub['sub2']==int(x))]
            try:subs_num = tmp['nums'].values[0]
            except:subs_num = 'subway_station中未能查到地铁'
        print(subs_num)
        subs_num_lst.append(subs_num)
    return subs_num_lst

In [96]:
zip1 = get_both_id(prob_matching,sub_info)
# for x,y in zip1:print(x,y)
subs_num_lst = get_subs_num(zip1,sub2sub)
print(len(subs_num_lst)==len(prob_matching))

71 59
5
217 71
3
38 38
0
220 134
17
1738 159
5
61 274
5
208 1433
3
1698 59
subway_station中未能查到地铁
72 221
3
68 168
7
146 125
10
71 236
22
244 61
10
1700 64
subway_station中未能查到地铁
41 123
16
223 95
18
1430 111
11
71 220
4
1048 70
9
126 149
8
244 *
subways中未查到地铁
268 94
2
71 1048
10
122 63
15
151 261
5
39 237
18
37 34
3
126 120
6
110 169
3
1051 1046
4
279 38
13
108 105
3
126 123
3
126 149
8
124 236
4
107 95
12
217 73
4
156 180
2
237 270
18
124 120
4
217 62
6
179 184
4
72 196
3
159 44
4
108 99
9
76 240
4
6 74
16
283 63
18
62 59
3
240 32
11
1051 257
30
35 247
23
35 121
8
99 38
22
270 134
4
279 75
7
244 278
9
237 122
5
1051 194
18
1051 169
13
1713 1430
9
206 32
31
125 122
3
124 1738
14
125 42
19
71 938
12
134 163
3
71 139
4
282 124
8
63 26
11
181 1746
19
1738 44
4
236 40
20
121 6
16
236 236
0
61 253
5
196 73
4
196 1048
12
176 265
7
152 17
4
258 233
17
108 103
5
61 121
12
132 106
17
171 134
6
169 135
7
96 180
6
200 129
3
96 23
9
35 39
4
98 95
3
238 183
27
184 181
3
95 134
8
42 205
40
201 129
4
17

In [103]:
prob_matching['subs_num'] = subs_num_lst
prob_matching.to_csv('df_al_block200_addnums.csv',index=0,na_rep='NA')`

In [ ]:
prob_matching['subs_num'] = subs_num_lst
prob_matching.to_csv('df_al_block200_addnums.csv',index=0,na_rep='NA')
zip1 = get_both_id(prob_matcing,sub_info)
# for x,y in zip2:print(x,y)
subs_num_lst = get_subs_num(zip1,sub2sub)
print(len(subs_num_lst)==len(prob_matching))